In [1]:
import pymssql
import pandas as pd
import string

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
conn = pymssql.connect(
    server='10.100.200.3',
    port='1433',
    user='pawel', 
    password='20Al3Mot@VP@weL22#', 
    database='SPEED',
    charset='ISO-8859-2')
cursor = conn.cursor()

In [4]:
def dataframe_from_query(given_cursor, given_query):
    given_cursor.execute(given_query)
    return_data = given_cursor.fetchall()
    dfr_columns = [item[0] for item in given_cursor.description]
    return_dataframe = pd.DataFrame(data=return_data, columns=dfr_columns)
    
    return return_dataframe
        

In [5]:
zlec_query = """
SELECT
       [ID_ZLECENIA]
      ,[NR_ZLECENIA]
      ,[SPEDYTOR] as 'spedytor'
      ,[ZA_MIEJSCE] as 'miejsce załadunku'
      ,[ZA_MIASTO] as 'miasto załadunku'
      ,[ZA_KRAJ] as 'kraj załadunku'
      ,[ZA_KOD] as 'kod załadunku'
      ,[TRASA] as 'trasa'
      ,[WY_MIEJSCE] as 'miejsce wyładunku'
      ,[WY_MIASTO] as 'miasto wyładunku'
      ,[WY_KRAJ] as 'kraj wyładunku'
      ,[WY_KOD] as 'kod wyładunku'
      ,[WY_DATA] as 'data wyładunku'
      ,[KWOTA_K] as 'przychód netto'
      ,[WALUTA_K] as 'waluta przychodu'
      ,[FAKTURA] as 'faktura'
      ,[FAKTURA_ZB_ID]
      ,[FAKTURA_ZB] as 'faktura zbiorcza'
      ,[NR_ZLECENIA_K]
      ,[KWOTA_P] as 'koszt przewoźnika'
      ,[WALUTA_P] as 'waluta koszt'
      ,[WALUTA_FV_P]
      ,[FAKTURA_K] as 'faktura przewoźnika'
      ,[FAKTURA_K_ZB] as 'faktura przew zbiorcza'
      ,[STATUS]
 FROM [SPEED].[dbo].[ZLECENIA]
 WHERE MONTH([ZA_DATA]) = 1
 AND YEAR([ZA_DATA]) = 2022
 AND [ZAFAKTUROWANE] = 1
 """

In [6]:
zlec_df = dataframe_from_query(cursor, zlec_query)

In [7]:
status_str = str(set(zlec_df.STATUS.to_list())).replace('{', '(').replace('}', ')')

status_query = f"""
  SELECT KOD, OPIS
  FROM [SPEED].[dbo].[STATUS]
  WHERE KOD IN {status_str}
  AND TYP = 'FT'
"""

status_df = dataframe_from_query(cursor, status_query).set_index('KOD')
zlec_df = zlec_df.join(status_df, on='STATUS')

In [13]:
relevant_zlec_ids = str(set(zlec_df.ID_ZLECENIA.to_list()))
relevant_zlec_ids = relevant_zlec_ids.replace('{', '(').replace('}', ')')

faktury_query = f"""
  SELECT * FROM FAKTURY WHERE ZLECENIE_ID IN {relevant_zlec_ids}"""

all_faktury_df = dataframe_from_query(cursor, faktury_query)

In [14]:
len(faktury_df.ZLECENIE_ID.unique())

521

In [15]:
numery_faktur_list = [el for el in zlec_df.faktura.to_list() if el not in ['', None]]
numery_faktur_str = str(set(numery_faktur_list)).replace('{', '(').replace('}', ')')

numery_faktur_query = f"""
  SELECT * FROM FAKTURY WHERE NUMER_FAKTURY IN {numery_faktur_str}"""

faktury_przych_df = dataframe_from_query(cursor, numery_faktur_query)

In [16]:
len(numery_faktur_list)

278

In [17]:
missing_faktury = [el for el in faktury_przych_df.NUMER_FAKTURY.to_list() if el not in faktury_df.NUMER_FAKTURY.to_list()]
missing_faktury

[]

In [18]:
len(faktury_przych_df)

278

In [19]:
faktury_cols = ['NETTO_PLN', 'NUMER_FAKTURY', 'ID_FAKTURY', 'ZLECENIE_ID']

In [ ]:
faktury_przych_df[faktury_cols]

### Noty

In [20]:
noty_cols = faktury_cols + ['NOTA', 'NOTA_UZNANIOWA']

nota_df = all_faktury_df[noty_cols].loc[all_faktury_df.NOTA == 1]

### Noty uznaniowe

In [21]:
nota_df['NETTO_PLN'].loc[nota_df.NOTA_UZNANIOWA == 1].apply(lambda x: x*(-1))

Series([], Name: NETTO_PLN, dtype: float64)

### Noty suma

In [22]:
nota_df = nota_df.groupby('ZLECENIE_ID')['NETTO_PLN'].sum()

In [25]:
faktury_df = all_faktury_df[faktury_cols].loc[all_faktury_df.NOTA == 0]

In [70]:
nota_df

ZLECENIE_ID
7884      27.27
7890    1094.30
7903     685.08
7922    2260.05
7961      27.60
7984    1358.10
7986      27.26
8026      27.30
8070      55.00
8098    2263.48
8101    1129.18
8157    2284.85
8180     633.85
8202    3634.56
8210    1131.85
8314      27.30
8316      55.00
8317      55.00
8334      27.30
8373      55.00
8441      27.60
Name: NETTO_PLN, dtype: float64

In [26]:
zlec_df.fillna('', inplace=True)
zlec_df.FAKTURA_ZB_ID = zlec_df.FAKTURA_ZB_ID.astype(str).replace('0', '')
for col in zlec_df.columns:
    if isinstance(zlec_df[col].iloc[1], str):
        zlec_df[col] = zlec_df[col].str.replace('Ľ', 'Ą')

In [27]:
zlec_cols = ['ID_ZLECENIA', 
                 'przychód netto', 
                 'waluta przychodu',
                 'faktura', 
                 'FAKTURA_ZB_ID', 
                 'NR_ZLECENIA_K', 
                 'koszt przewoźnika', 
                 'waluta koszt', 
                 'WALUTA_FV_P', 
                 'faktura przewoźnika', 
                 'faktura przew zbiorcza']

In [28]:
len(zlec_df.FAKTURA_ZB_ID.unique())

21

In [29]:
len(zlec_df.faktura.unique())

279

In [30]:
zlec_df[zlec_cols + ['OPIS']].loc[zlec_df['waluta przychodu'] != zlec_df['waluta koszt']].sample(1)

,ID_ZLECENIA,przychód netto,waluta przychodu,faktura,FAKTURA_ZB_ID,NR_ZLECENIA_K,koszt przewoźnika,waluta koszt,WALUTA_FV_P,faktura przewoźnika,faktura przew zbiorcza,OPIS
539,8387,2730.0,PLN,00137/FV/01/2022/MS,,FR/2022/01/25/2JNRAL4500641439,350.0,EUR,EUR,54/2022,,Zafakturowano


In [32]:
zlec_df.faktura.value_counts().unique()

array([391,   1], dtype=int64)

In [33]:
unique_ids = zlec_df.ID_ZLECENIA.to_list()

1. Pobieram Zlecenia, patrzę według daty Wyładunku wybrany miesiąc
2. Pobieram Faktury na podstawie ID zleceń
3. Faktury Przychód na podstawie zleceń
4. Faktury Koszt na podstawie Zleceń
5. Faktury Nota na podstawie Zleceń

In [34]:
numery_faktur_przych_list = [el for el in zlec_df.faktura.to_list() if el != '']

In [35]:
przych_faktury_df = all_faktury_df.loc[all_faktury_df.NUMER_FAKTURY.isin(numery_faktur_przych_list)].set_index('NUMER_FAKTURY')

In [36]:
przych_faktury_df['NETTO_PLN']

NUMER_FAKTURY
00008/FV/01/2022/MS    7317.92
00031/FV/01/2022/MS    4543.01
00030/FV/01/2022/MS    4345.02
00029/FV/01/2022/MS    4359.46
00028/FV/01/2022/MS    4359.46
                        ...   
00024/FV/02/2022/MS     683.25
00025/FV/02/2022/MS     455.50
00187/FV/01/2022/MS    3542.00
00188/FV/01/2022/MS    3058.30
00191/FV/01/2022/MS    6781.40
Name: NETTO_PLN, Length: 278, dtype: float64

In [37]:
numery_faktur_koszt_list = [el for el in zlec_df['faktura przewoźnika'].to_list() if el != '']

In [38]:
koszt_faktury_df = all_faktury_df.loc[all_faktury_df.NUMER_FAKTURY.isin(numery_faktur_koszt_list)].set_index('NUMER_FAKTURY')

In [39]:
koszt_faktury_df['NETTO_PLN']

NUMER_FAKTURY
2/2022            6860.55
FV/62/01/2021     4084.12
FV/124/01/2022    4070.59
FV/36/01/2022     4070.59
FV/126/01/2022    4070.59
                   ...   
8/TR/2022         1010.00
FS/22/2/66        1977.23
F-WAW-22-01280    9828.30
F-WAW-22-01281     457.13
F-WAW-22-01282     457.13
Name: NETTO_PLN, Length: 481, dtype: float64

In [40]:
saldo_cols = ['przychód netto', 'waluta przychodu', 'faktura', 'FAKTURA_ZB_ID', 'faktura przewoźnika', 'faktura przew zbiorcza', 'koszt przewoźnika', 'waluta koszt']

In [41]:
len(zlec_df)

669

In [43]:
zlec_df.faktura.loc[zlec_df.faktura != ''].value_counts()

00124/FV/01/2022/MS     1
00024/FV/01/2022/MS     1
00089/FV/01/2022/ALB    1
00017/FV/01/2022/ALB    1
00053/FV/01/2022/ALB    1
                       ..
00047/FV/01/2022/ALB    1
00027/FV/01/2022/ALB    1
00004/FV/01/2022/MS     1
00007/FV/02/2022/MS     1
00178/FV/01/2022/MS     1
Name: faktura, Length: 278, dtype: int64

In [47]:
zlec_df[saldo_cols].loc[zlec_df.faktura != ''].join(przych_faktury_df['NETTO_PLN'], how='left', on = 'faktura')

,przychód netto,waluta przychodu,faktura,FAKTURA_ZB_ID,faktura przewoźnika,faktura przew zbiorcza,koszt przewoźnika,waluta koszt,NETTO_PLN
0,1900.0,EUR,00020/FV/02/2022/ALB,,,,1800.0,EUR,8705.80
1,1600.0,EUR,00008/FV/01/2022/MS,,2/2022,,1500.0,EUR,7317.92
2,990.0,EUR,00031/FV/01/2022/MS,,FV/62/01/2021,,890.0,EUR,4543.01
3,950.0,EUR,00029/FV/01/2022/MS,,FV/36/01/2022,,890.0,EUR,4359.46
4,950.0,EUR,00028/FV/01/2022/MS,,FV/126/01/2022,,890.0,EUR,4359.46
...,...,...,...,...,...,...,...,...,...
652,330.0,EUR,00002/FV/02/2022/MS,,3/02/2022,,300.0,EUR,1517.41
654,650.0,EUR,00029/FV/02/2022/ALB,,31/2022,,600.0,EUR,2988.83
657,900.0,EUR,00066/FV/02/2022/MS,,12/2022,,750.0,EUR,4095.00
658,360.0,EUR,00010/FV/02/2022/MS,,3/02/2022,,290.0,EUR,1655.35


In [50]:
len(zlec_df.loc[zlec_df['faktura przewoźnika'] != ''])

480

In [49]:
zlec_df[saldo_cols].loc[zlec_df['faktura przewoźnika'] != ''].join(koszt_faktury_df['NETTO_PLN'], how='left', on = 'faktura przewoźnika')

,przychód netto,waluta przychodu,faktura,FAKTURA_ZB_ID,faktura przewoźnika,faktura przew zbiorcza,koszt przewoźnika,waluta koszt,NETTO_PLN
1,1600.00,EUR,00008/FV/01/2022/MS,,2/2022,,1500.0,EUR,6860.55
1,1600.00,EUR,00008/FV/01/2022/MS,,2/2022,,1500.0,EUR,3197.04
2,990.00,EUR,00031/FV/01/2022/MS,,FV/62/01/2021,,890.0,EUR,4084.12
3,950.00,EUR,00029/FV/01/2022/MS,,FV/36/01/2022,,890.0,EUR,4070.59
4,950.00,EUR,00028/FV/01/2022/MS,,FV/126/01/2022,,890.0,EUR,4070.59
...,...,...,...,...,...,...,...,...,...
657,900.00,EUR,00066/FV/02/2022/MS,,12/2022,,750.0,EUR,3412.50
658,360.00,EUR,00010/FV/02/2022/MS,,3/02/2022,,290.0,EUR,1379.46
658,360.00,EUR,00010/FV/02/2022/MS,,3/02/2022,,290.0,EUR,1333.48
659,1085.72,PLN,,13583,8/TR/2022,,1010.0,PLN,1010.00


In [ ]:
zlec_df[zlec_cols].loc[zlec_df.WALUTA_FV_P != 'PLN']

In [51]:
zlec_query_1 = """
 SELECT * 
 FROM [SPEED].[dbo].[ZLECENIA]
 WHERE MONTH([ZA_DATA]) = 1
 AND YEAR([ZA_DATA]) = 2022
 AND [ZAFAKTUROWANE] = 1
 """

In [52]:
zlec_df_1 = dataframe_from_query(cursor, zlec_query_1)

In [53]:
zlec_df_1.columns

Index(['ID_ZLECENIA', 'MYJNIA', 'ZLECENIE_ID', 'BEZ_SFM', 'SFM_STATUS',
       'SFM_VER', 'SFM_L_ZA_WY', 'SFM_WYSL', 'MobileStatus', 'MobileSequence',
       ...
       'CREATE_USER_NAME', 'CREATE_DATE', 'CREATE_TIME', 'MODIFY_ID_USER',
       'MODIFY_USER_NAME', 'MODIFY_DATE', 'MODIFY_TIME', 'DATA_KONTR_WYKAZ_WL',
       'WYNIK_KONTR_WYKAZ_WL', 'REQUEST_ID'],
      dtype='object', length=347)

In [56]:
zlec_fakt_col = [el for el in list(all_faktury_df.columns) if 'ZLEC' in el]

In [57]:
zlec_fakt_col

['ZLECENIE_ID',
 'ZLECENIE',
 'ZLECENIE_C_ID',
 'ZLECENIE_C',
 'ZLECENIE_M_ID',
 'ZLECENIE_M',
 'ZLECENIE_RAIL_ID',
 'ZLECENIE_RAIL',
 'ZLECENIE_ZB_ID',
 'ZLECENIE_ZB',
 'NR_ZLEC_K',
 'W_ZLECENIE',
 'W_ZLECENIE_ID']

In [64]:
faktury_zlec_counts = all_faktury_df.ZLECENIE_ID.value_counts()

In [69]:
faktury_zlec_counts.loc[faktury_zlec_counts == 1]

8426    1
8425    1
8396    1
8398    1
8211    1
       ..
7862    1
7867    1
7868    1
7869    1
8192    1
Name: ZLECENIE_ID, Length: 267, dtype: int64

In [63]:
all_faktury_df[faktury_cols + ['NOTA', 'NOTA_UZNANIOWA']].loc[all_faktury_df.ZLECENIE_ID == 8202]

,NETTO_PLN,NUMER_FAKTURY,ID_FAKTURY,ZLECENIE_ID,NOTA,NOTA_UZNANIOWA
448,6414.94,021/2022,13513,8202,0,0
449,8018.68,00022/FV/02/2022/MS,13653,8202,0,0
450,1827.88,01/01/2022,13755,8202,1,0
451,1806.68,00036/N/2022/OK/MS,13788,8202,1,0
